## SMART CITY PROJECT
![smart-city-labos.jpeg](attachment:smart-city-labos.jpeg)
#### Project context
<dl>

  <dd>Our project concerns the analysis for the improvement of the management of all kinds of anomalies declared in Paris. 
We will proceed to these analyses by using a dataset that we extracted from the site 'site name'.  All of our analyses on this dataset will be done from January 1, 2019 - April 2022. 

To give more sense to our analyses, we will also use a dataset extracted from the site 'site name', which gives us access to the weather data of the city of Paris. 

We will proceed to the cleaning, and to the analyses of each dataset separately and independently of the other one to know the extent of the information which we can draw from it. 

Finally, we will concatenate our two datasets to look for correlations between the weather data and the anomalies recorded in the city of Paris. 

At the end of our study we will be able to bring recommendations to the city of Paris to improve the management and correction of anomalies according to several criteria. 

Enjoy ! </dd>
    <dl>
### Presented to you by: 
Constantin Bogdanas & Anissa Hacene

In [2]:
from zipfile import ZipFile, Path
import pandas as pd
import plotly.express as px
import folium
import seaborn as sns
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster 

In [3]:
def get_data(path: str) -> pd.DataFrame:
    '''
    Get data from zip file
    '''
    with ZipFile(path) as myzip:
        data = myzip.open(path.split('.zip')[0])

    df = pd.read_csv(data, sep =';')
    return df

In [4]:
df_2021_2022 = get_data("DMR_2021_2022.csv.zip")
df_2020 = get_data("DMR_2020.csv.zip")
df_2019 = get_data("DMR_2019.csv.zip")

In [5]:
df_2021_2022[['X', 'Y']] = df_2021_2022['geo_point_2d'].str.split(',', expand=True)

In [6]:
df_2021_2022 = df_2021_2022.drop(['ID DECLARATION', 'VILLE', 'ANNEE DECLARATION', 'MOIS DECLARATION',
         'OUTIL SOURCE', 'ID_DMR', 'geo_shape', 'geo_point_2d', 'CONSEIL DE QUARTIER'], axis = 1)

In [7]:
df_2019_2020 = pd.concat([df_2019, df_2020])
df_2019_2020 = df_2019_2020.drop(['OBJECTID', 'ID_DMR', 'VILLE' ,'ANNEEDECL' ,'MOISDECL' ,'NUMERO','PREFIXE', 'CONSEILQUARTIER',  ], axis = 1)


In [8]:
df_2019_2020.rename(columns = {'TYPE':'TYPE DECLARATION', 'SOUSTYPE':'SOUS TYPE DECLARATION', 'CODE_POSTAL':'CODE POSTAL', 'DATEDECL':'DATE DECLARATION'}, inplace = True)
df_2019_2020.rename(columns = {'Y':'LONGITUDE', 'X':'LATITUDE'}, inplace = True)
df_2021_2022.rename(columns = {'X':'LONGITUDE', 'Y':'LATITUDE'}, inplace = True)
df_final = pd.concat([df_2021_2022, df_2019_2020])


In [9]:
df_final.head()

,TYPE DECLARATION,SOUS TYPE DECLARATION,ADRESSE,CODE POSTAL,ARRONDISSEMENT,DATE DECLARATION,INTERVENANT,LONGITUDE,LATITUDE
0,Mobiliers urbains,Collecte des déchets:Trilib' dégradé,"18 Rue de Mazagran, 75010 PARIS",75010,10,2021-01-08,TRILIB',48.8707730016144,2.352240998304316
1,Objets abandonnés,Autres objets encombrants abandonnés,"58 Rue de l'Aqueduc, 75010 PARIS",75010,10,2021-01-08,Ramen en tant que prestataire de DansMaRue,48.88284700351721,2.365264406238782
2,Objets abandonnés,Autres objets encombrants abandonnés,"106 Rue des Amandiers, 75020 PARIS",75020,20,2021-01-08,Ramen en tant que prestataire de DansMaRue,48.86777999827195,2.388730996327768
3,"Graffitis, tags, affiches et autocollants","Graffitis sur mur, façade sur rue, pont","14 Rue Carrière-Mainguet, 75011 PARIS",75011,11,2021-01-08,graffitis,48.856599995660474,2.387607803079172
4,Objets abandonnés,Cartons,"1 Square Alfred Capus, 75016 PARIS",75016,16,2021-01-08,DPE-STPP-DT,48.85093700329557,2.259631402809926


In [10]:
arrondissement_count = df_final["ARRONDISSEMENT"].value_counts()

fig = px.bar(arrondissement_count, x=arrondissement_count.index, y='ARRONDISSEMENT',
             hover_data=[arrondissement_count.index, 'ARRONDISSEMENT'], color='ARRONDISSEMENT',
             labels={'index': "District number",'ARRONDISSEMENT':'District'}, height=400)
fig.update_xaxes(type='category')
fig.show()
# USE INDEX AS X AXIS

In [11]:
fig = px.pie(arrondissement_count, values='ARRONDISSEMENT',
            names=arrondissement_count.index, 
            title='Pie chart of anomalies by district',
            color='ARRONDISSEMENT',
            color_discrete_sequence=px.colors.cyclical.IceFire,
            labels={'index': "Number", 'ARRONDISSEMENT': 'Anomalies' }, height=400)
fig.update_layout(width=1000,
    height=700,
    legend_title_text='District'
)

fig.show()